<a href="https://colab.research.google.com/github/anujatayal/Colab-Notebooks/blob/main/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=ee03ffce3e12b64bf96da9c1ec0601b6bc660f67a16a36d9293c186c0133d4e3
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=b87bc5da244905a3f5e380c3d81791588058539ca3f5d5c5019793aaf13600fb
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=889a11296bd9895ea8ec845de4d50a52062c0145a4f9b65e64bf2411f2c9ece5
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 3.4MB/s 
TensorFlow Version: 2.3.0
Hub versi

In [ ]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)
#https://androidkt.com/simple-text-classification-using-bert-in-tensorflow-keras-2-0/

In [ ]:
MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,name="segment_ids")

In [ ]:
#input token ids is tokenizer converts tokens using vocab file.
#input masks are either 0 or 1. 1 for useful tokens, 0 for padding.
#segment ids are either 0 or 1. For 2 text training: 0 for the first one, 1 for the second one.


In [ ]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [ ]:
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
#pooled_output representations the entire input sequences and sequence_output representations each input token in the context.



In [ ]:
FullTokenizer=bert.bert_tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "anujatayal" # username from the json file
os.environ['KAGGLE_KEY'] = "4566267aec1fb886b205af0ce951f202" # key from the json file
/content/sample_data/train.csv!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 78% 41.0M/52.6M [00:01<00:00, 21.8MB/s]
100% 52.6M/52.6M [00:01<00:00, 33.3MB/s]


In [ ]:
pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.9)


In [ ]:
import pandas as pd

df=pd.read_csv('/content/sample_data/train.csv',engine='python',error_bad_lines=False)

df = df.sample(frac=1)

train_sentences = df1["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = df1[list_classes].values

Skipping line 31600: unexpected end of data


In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
15497,28e41d194c823996,"""\n\n Harassment \n\nPlease do not Out contrib...",0,0,0,0,0,0
7962,1534e164eee103a1,"Hey, dumbfuck, quit fucking around with Wikipe...",1,0,1,0,1,0
20723,36b6cd63eb7cac9f,The comments that have been made seem to make ...,0,0,0,0,0,0
16565,2baf211e95de7aac,"Revert Again -Yes, LGagnon is well-known throu...",0,0,0,0,0,0
16367,2b22e0ffb5b65ece,"""\n""""Vandalism""""? Sorry, mister, but I haven't...",1,0,0,0,0,0


In [ ]:
def create_single_input(sentence,MAX_LEN):
  stokens = tokenizer.tokenize(sentence)
  stokens = stokens[:MAX_LEN]
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
  return ids,masks,segments

def create_input_array(sentences):
  input_ids, input_masks, input_segments = [], [], []
  for sentence in tqdm(sentences,position=0, leave=True):
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)

  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [ ]:
x = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
x = tf.keras.layers.Dropout(0.2)(x)
out = tf.keras.layers.Dense(6, activation="sigmoid", name="dense_output")(x)

model = tf.keras.models.Model(
      inputs=[input_word_ids, input_mask, segment_ids], outputs=out)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
inputs=create_input_array(train_sentences)

model.fit(inputs,train_y,epochs=1,batch_size=32,validation_split=0.2,shuffle=True)


100%|██████████| 5000/5000 [00:05<00:00, 937.95it/s]


125/125 [==============================] - 132s 1s/step - loss: 0.1663 - accuracy: 0.6768 - val_loss: 0.1679 - val_accuracy: 0.9950


In [ ]:
test_df=pd.read_csv("/content/sample_data/test.csv")
test_sentences = test_df["comment_text"].fillna("CVxTz").values
test_inputs=create_input_array(test_sentences[110:150])
print(model.predict(test_inputs))

100%|██████████| 40/40 [00:00<00:00, 603.26it/s]


[[0.05256845 0.00570976 0.01252703 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256845 0.00570976 0.01252703 0.00219777 0.03246804 0.0058945 ]
 [0.05256847 0.00570976 0.01252703 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252703 0.00219777 0.03246803 0.0058945 ]
 [0.05256845 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256845 0.00570976 0.01252703 0.00219777 0.03246803 0.0058945 ]
 [0.05256845 0.00570976 0.01252703 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252702 0.00219777 0.03246803 0.0058945 ]
 [0.05256847 0.00570976 0.01252703 0.00219777 0.03246804 0.0058945 ]
 [0.05256847 0.00570976 0.01252703 0.00219777 0.03246804 0.0058945 ]
 [0.05256847 0.00570976 0.01252702

In [ ]:
df1=df[:5000]

In [ ]:
len(df1)

5000